In [2]:
## GERAR CASOS de guillain barre e trombose
#  conferir manualmente
#  fazer a análise no scrip_incidencia_geral

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Chamar banco evento_advero
#df = pd.read_excel('Relatorios/evento_adverso.xlsx')
df = pd.read_csv('eapv_08022022_09h23.csv', sep=';', encoding='utf-8')
banco_completo = df.shape[0]

In [15]:
# Transformando ['Número da Notificação'] para string
df['Número da Notificação'] = df['Número da Notificação'].astype(str)
# puxar arquivo canceladas
df_canceladas = pd.read_csv('eapv_canceladas_08022022.csv', sep=';', encoding='utf-8')
# df_canceladas passa a ser apenas a coluna 'Número da Notificação' e em string
df_canceladas = df_canceladas[['Número da Notificação']].astype(str)
# Transformando o banco df_canceladas para uma lista notif_canceladas
banco_canceladas = df_canceladas.shape[0]
notif_canceladas = df_canceladas['Número da Notificação'].to_list()
# TIRANDO AS EXCLUÍDAS DO BANCO DE DADOS 
df = df[~df['Número da Notificação'].str.contains('(?i)' + '|'.join(notif_canceladas), na=False)]
# conferir
print(f'Banco sem as canceladas: {df.shape[0]}')
print(f'Banco completo: {banco_completo} - canceladas: {banco_canceladas} = {banco_completo - banco_canceladas}')
# Verificar se ficou alguma cancelada e excluir do banco
df[df['Situação da Notificação'] == 'Cancelado']
df = df[~df['Situação da Notificação'].str.contains('Cancelado', na=False)]

Banco sem as canceladas: 16381
Banco completo: 17153 - canceladas: 772 = 16381


In [21]:
df_enc = df[['Situação da Notificação']]
df_enc.loc[df_enc['Situação da Notificação'].str.contains('Em Avaliação'), 'Situação da Notificação'] = 'Aberto'
df_enc['Total'] = 1
df_enc = df_enc.groupby(by=['Situação da Notificação']).sum().reset_index()
df_enc['%'] = ((df_enc['Total']/df.shape[0])*100).round(0)
df_enc

,Situação da Notificação,Total,%
0,Aberto,8131,50.0
1,Encerrado,8247,50.0


# Guillain Barre

In [10]:
df_guillain = df.copy()
# pesquisa 'guill' em todo o dataframe
df_guillain = df.loc[df.apply(lambda row: row.astype(str).str.contains('(?i)guill').any(), axis=1)]
# Criando nova coluna para identificação
df_guillain['VACINA'] = ''
df_guillain['CLASSIFICAÇÃO'] = 'Indefinido'
# criar excel para conferencia manial
df_guillain.to_excel('Casos/guillain_barre_boletim_06.xlsx', index=False)

# Trombose

In [6]:
df_trombose = df.copy()
# Fazer um lista com os nomes para pesquisa 
trombose = ['trombo', 'embolia']
# Pesquisa {tromboses} em todo o dataframe
df_trombose = df_trombose.loc[df_trombose.apply(lambda row: row.astype(str).str.contains('(?i)' + '|'.join(trombose)).any(), axis=1)]
# Criando nova coluna para identificação
df_trombose['VACINA'] = ''
df_trombose['CLASSIFICAÇÃO'] = 'Indefinido'
# criar excel para conferencia manial
df_trombose.to_excel('Casos/trombose_boletim_06.xlsx', index=False)

In [7]:
df.columns.to_list()

['Número da Notificação',
 'Estado da Notificação',
 'Município da Notificação',
 'Idade Evento',
 'Verssão MedDRA',
 'Você está notificando um evento adverso ou erro de imunização ocorrido em uma criança que foi exposta à vacina através da vacinação da mãe durante a gestação ou pelo aleitamento materno?',
 'Data da Notificação',
 'Criança em aleitamento materno, no momento da vacinação?',
 'Situação da Notificação',
 'Profissional de Segurança',
 'Telefone de Contato',
 'Tem CPF?',
 'Número (ou SN para Sem Número)',
 'Data de Nascimento',
 'Bairro',
 'Telefone Celular',
 'Estado de Residência',
 'CPF',
 'Município de Residência',
 'Passaporte',
 'Nome Completo da Mãe',
 'Estrangeiro',
 'CNS',
 'Logradouro',
 'É profissional de saúde?',
 'Complemento',
 'Raça/Cor',
 'Sexo',
 'Nome Completo',
 'Pais de origem',
 'CBO',
 'CEP',
 'Etnia',
 'Comunidade/Povo Tradicional?',
 'Comunidade/Povo Tradicional',
 'Imunobiológico (vacina)',
 'Código Imunobiológico',
 'Nome do Fabricante',
 'Relação 

In [16]:
curioso = df.loc[df.apply(lambda row: row.astype(str).str.contains('(?i)Natanael').any(), axis=1)]
curioso.to_excel('curioso.xlsx')